In [2]:
# %% [code]
import pickle
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display
import asyncio

num_person = "both"

pkl_location_h1h2 = f'/Users/emilydu/Code/Code_lingustic/Data/f0_h1h2_xy_pkl_save/{num_person}/h1h2_xy.pkl'
pkl_location_f0   = f'/Users/emilydu/Code/Code_lingustic/Data/f0_h1h2_xy_pkl_save/{num_person}/f0_xy.pkl'

with open(pkl_location_h1h2, 'rb') as f:
    h1h2_dict = pickle.load(f)
with open(pkl_location_f0, 'rb') as f:
    f0_dict = pickle.load(f)

tones = list(h1h2_dict.keys())  # e.g., ['qu', 'ru', 'shang', 'yang', 'yin']

# --------------------------
# GLOBAL CONTROLS
# --------------------------
global_max_frames = max(len(h1h2_dict[t]['x']) for t in tones) - 1
frame_slider = widgets.IntSlider(
    min=0, max=global_max_frames, step=1, value=0, description="Frame:"
)
start_button = widgets.Button(description="Start Animation", button_style='success')
stop_button  = widgets.Button(description="Stop Animation",  button_style='warning')
speed_slider = widgets.IntSlider(value=5, min=1, max=10, step=1, description="Speed:")

# --------------------------
# FIGURES & METERS
# --------------------------
figs = {}
h1h2_progress = {}
f0_progress   = {}
h1h2_labels   = {}
f0_labels     = {}

y_span  = [-20, 20]
x_range = [-10, 10]

def make_vertical_meter(bar_color='#1f77b4'):
    """
    Create a FloatProgress widget that fills from bottom to top.
    To achieve this, we rotate it -90 degrees (counterclockwise),
    so that what was originally left-to-right becomes bottom-to-top.
    """
    return widgets.FloatProgress(
        value=0,
        layout=widgets.Layout(
            width='30px',
            height='150px',
            transform='rotate(-90deg)‘
        ),
        style={'bar_color': bar_color}
    )

for tone in tones:
    h1h2_arr = np.array(h1h2_dict[tone]['y'])
    f0_arr   = np.array(f0_dict[tone]['y'])

    fig = go.FigureWidget()
    fig.add_scatter(x=[0, 0], y=y_span, mode='lines',
                    line=dict(color='blue', width=4), showlegend=False)
    fig.add_scatter(x=[0, 0], y=y_span, mode='lines',
                    line=dict(color='red', width=4), showlegend=False)
    fig.update_layout(
        title=f"{tone}",
        xaxis=dict(visible=False, range=x_range),
        yaxis=dict(range=y_span),
        showlegend=False,
        margin=dict(l=0, r=0, t=30, b=0)
    )
    figs[tone] = fig

    h1h2_min_val, h1h2_max_val = -10, 10
    f0_min_val, f0_max_val     = 100, 300

    h_meter = make_vertical_meter('#1f77b4')
    h_meter.min = h1h2_min_val
    h_meter.max = h1h2_max_val
    h_label = widgets.Label(value="0.00")

    f_meter = make_vertical_meter('#d62728')
    f_meter.min = f0_min_val
    f_meter.max = f0_max_val
    f_label = widgets.Label(value="0.00")

    h1h2_progress[tone] = h_meter
    f0_progress[tone]   = f_meter
    h1h2_labels[tone]   = h_label
    f0_labels[tone]     = f_label

# --------------------------
# HELPER FUNCTIONS
# --------------------------
def f0_to_color(current_f0, f0_min, f0_max):
    if f0_max == f0_min:
        return "rgb(127,0,127)"
    t = (current_f0 - f0_min) / (f0_max - f0_min)
    r = int(t * 255)
    g = 0
    b = int((1 - t) * 255)
    return f"rgb({r},{g},{b})"

def update_all_figures(change=None):
    global_frame = frame_slider.value
    for tone in tones:
        time_arr = np.array(h1h2_dict[tone]['x'])
        h1h2_arr = np.array(h1h2_dict[tone]['y'])
        f0_arr   = np.array(f0_dict[tone]['y'])

        if len(time_arr) == 0 or len(h1h2_arr) == 0 or len(f0_arr) == 0:
            continue

        frame = min(global_frame, len(time_arr)-1, len(h1h2_arr)-1, len(f0_arr)-1)
        current_time = time_arr[frame]
        current_h1h2 = h1h2_arr[frame]
        current_f0   = f0_arr[frame]

        separation = current_h1h2 if current_h1h2 > 0 else 0
        left_x  = -separation/2
        right_x =  separation/2

        f0_min_val = float(np.min(f0_arr))
        f0_max_val = float(np.max(f0_arr))
        if f0_min_val > f0_max_val:
            f0_min_val, f0_max_val = f0_max_val, f0_min_val
        color = f0_to_color(current_f0, f0_min_val, f0_max_val)

        fig = figs[tone]
        with fig.batch_update():
            fig.data[0].x = [left_x, left_x]
            fig.data[0].y = y_span
            fig.data[1].x = [right_x, right_x]
            fig.data[1].y = y_span
            fig.data[0].line.color = color
            fig.data[1].line.color = color
            fig.layout.title = f"{tone}: Frame {frame} | Time: {current_time:.2f}"

        h1h2_progress[tone].value = current_h1h2
        f0_progress[tone].value   = current_f0
        h1h2_labels[tone].value   = f"{current_h1h2:.2f}"
        f0_labels[tone].value     = f"{current_f0:.2f}"

# --------------------------
# ANIMATION LOGIC
# --------------------------
is_animating = False
animation_task = None

async def animate_frames():
    global is_animating
    is_animating = True
    total_frames = frame_slider.max + 1
    while is_animating:
        new_val = frame_slider.value + 1
        if new_val >= total_frames:
            new_val = 0
        frame_slider.value = new_val
        fps = (speed_slider.value / 10) * total_frames
        delay = 1 / fps
        await asyncio.sleep(delay)

def start_animation(_):
    global animation_task, is_animating
    if animation_task is None or animation_task.done():
        is_animating = True
        animation_task = asyncio.ensure_future(animate_frames())

def stop_animation(_):
    global is_animating, animation_task
    is_animating = False
    if animation_task is not None:
        animation_task.cancel()
        animation_task = None

frame_slider.observe(update_all_figures, names='value')
start_button.on_click(start_animation)
stop_button.on_click(stop_animation)

update_all_figures()

# --------------------------
# BUILD UI
# --------------------------
tone_boxes = []
for tone in tones:
    h_box = widgets.VBox([
        widgets.Label(f"{tone} h1h2:"),
        h1h2_progress[tone],
        h1h2_labels[tone],
    ])
    f_box = widgets.VBox([
        widgets.Label(f"{tone} f0:"),
        f0_progress[tone],
        f0_labels[tone],
    ])
    meter_row = widgets.HBox([h_box, f_box])
    tone_box  = widgets.HBox([figs[tone], meter_row])
    tone_boxes.append(tone_box)

global_controls = widgets.HBox([frame_slider, start_button, stop_button, speed_slider])
final_ui = widgets.VBox([global_controls] + tone_boxes)
display(final_ui)

SyntaxError: unterminated string literal (detected at line 55) (3387584497.py, line 55)